<a href="https://colab.research.google.com/github/sorendunn/magicoder/blob/main/MagiCoderDataGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Dependencies

In [36]:
!git clone https://github.com/ise-uiuc/magicoder.git

fatal: destination path 'magicoder' already exists and is not an empty directory.


In [37]:
%cd magicoder/

/content/magicoder/magicoder


In [39]:
!pip install datasets

In [10]:
!cp -r src/magicoder magicoder

In [19]:
%cd ..

/content/magicoder


In [20]:
!cp magicoder/generate_data.py generate_data.py

In [13]:
%cd magicoder

/content/magicoder/magicoder


In [22]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00


In [24]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.3 MB/s eta 0:00:00


In [96]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["START_INDEX_OF_RAW_DATA"] = "1"
os.environ["MAX_DATA_TO_GENERATE"] = "1500"
os.environ["MBPP_PATH"] = "/content/magicoder/mbpp_dataset.jsonl"
os.environ["DS1000_PATH"] = "/content/magicoder/ds1000_data"
os.environ["PREPROCESS_OUTPUT_PATH"] = "/content/magicoder/preprocessed.jsonl "

In [20]:
!cp -r magicoder src/magicoder/magicoder

In [41]:
from huggingface_hub import notebook_login

notebook_login()

In [58]:
!cp /content/*.py /content/magicoder/src/magicoder/

In [60]:
!cp /content/magicoder/src/magicoder/find_substrings_scot.py /content/magicoder/src/magicoder/decontamination/find_substrings_scot.py

### Load in Required Datasets

In [ ]:
!unzip ../ds1000_data.zip

In [ ]:
from datasets import load_dataset
import json

# Load the mbpp dataset
dataset = load_dataset("mbpp")

# Specify the path for the JSONL file
jsonl_file_path = "mbpp_dataset.jsonl"

# Open the file in write mode
with open(jsonl_file_path, 'w', encoding='utf-8') as file:
    # Iterate through the dataset
    for item in dataset['test']:  # Assuming you want to save the 'train' split.
        # Convert the dictionary to a JSON string
        json_str = json.dumps(item)
        # Write the JSON string to the file, followed by a newline character to separate the records
        file.write(json_str + '\n')

print(f"Dataset saved to {jsonl_file_path}")

## Generate Base Instruction-Tuning Data

In [91]:
!python src/magicoder/generate_data.py \
  --seed_code_start_index ${START_INDEX_OF_RAW_DATA} \
  --max_new_data ${MAX_DATA_TO_GENERATE} \
  --data_dir python \
  --tag python

train[:10000]
Parameter 'fn_kwargs'={'args': Args(seed_code_start_index=0, max_new_data=10, continue_from=None, seed=976, temperature=0.0, model='gpt-3.5-turbo-1106', model_max_tokens=8192, max_new_tokens=2500, min_lines=1, max_lines=15, chunk_size=1000, dataset_name='Nan-Do/code-search-net-python', data_dir='python', max_considered_data=10000, tag='python')} of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map: 100% 10000/10000 [00:00<00:00, 32979.35 examples/s]
Map: 100% 10000/10000 [00:03<00:00, 3123.83 examples/s]
Saving to data-python-77804-0_10-20240319_014446.jsonl
  0% 0

In [92]:
!python src/magicoder/clean_data.py --data_files /content/magicoder/data-python-77804-0_10-20240319_014446.jsonl --output_file /content/magicoder/cleaned_data_test.jsonl

Filtering same seed, problem, and solution: 100% 10/10 [00:00<00:00, 17091.70it/s]
After filtering: 10 -> 10
/content/magicoder/src/magicoder/clean_data.py:194: UserWarning: In practice, filtering data whose solution copies the problem does not help much.So we disabled it. But this conclusion remains to be verified.
  warnings.warn(


In [93]:
!python -m magicoder.decontamination.find_substrings \
  --dataset_name "json" \
  --output_file /content/magicoder/decontaminated_test.jsonl \
  --output_dir /content/magicoder/decontaminated_test \
  --columns problem solution \
  --data_files /content/magicoder/cleaned_data_test.jsonl

Ignoring: ['']
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for codeparrot/apps contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/codeparrot/apps
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
num strings from mbpp_docstrings: 500
num strings from mbpp_solutions: 500
num strings from human_eval_docstrings: 164
num strings from human_eval_solutions: 161
num strings from apps_docstrings: 5000
num strings from gsm8k_questions: 1319
num strings from ds_1000_prompts: 1000
Generating train split: 10 examples [00:00, 699.59 examples/s]
num_proc must be <= 10. Reducing num_proc to 10 for dataset of size 10.
Map (num_proc=10): 100% 10/10 [00:02<00:00,  4.95 examples/s]
Number of samples in 

In [94]:
!python src/magicoder/preprocess_data.py \
  --dataset_path json \
  --data_files /content/magicoder/decontaminated_test.jsonl \
  --output_file ${PREPROCESS_OUTPUT_PATH} \
  --key src-instruct

Generating train split: 10 examples [00:00, 1998.81 examples/s]
Map: 100% 10/10 [00:00<00:00, 788.25 examples/s]


## Generate Code Search Net Training Data

In [105]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["START_INDEX_OF_RAW_DATA"] = "0"
os.environ["MAX_DATA_TO_GENERATE"] = "150"
os.environ["MBPP_PATH"] = "/content/magicoder/mbpp_dataset.jsonl"
os.environ["DS1000_PATH"] = "/content/magicoder/ds1000_data"
os.environ["PREPROCESS_OUTPUT_PATH"] = "/content/magicoder/preprocessed_code_search_net.jsonl"
os.environ["PATH_TO_DATA_FILE"] = "/content/magicoder/CHANGE.jsonl" ## May need to change
os.environ["CLEANING_OUTPUT_PATH"] = "/content/magicoder/cleaned_data_code_search_net.jsonl"
os.environ["DECONTAM_OUTPUT_PATH"] = "/content/magicoder/decontaminated_code_search_net.jsonl"
os.environ["OUTPUT_DIR"] = "/content/magicoder/decontaminated_code_search_net"
os.environ["PATH_TO_DATA_FILE"] = "/content/magicoder/cleaned_code_search_net.jsonl"
os.environ["PREPROCESS_OUTPUT_PATH"] = "/content/magicoder/preprocessed_code_search_net.jsonl"

In [98]:
%cd /content/magicoder

/content/magicoder


In [109]:
!python src/magicoder/generate_data.py \
  --seed_code_start_index ${START_INDEX_OF_RAW_DATA} \
  --max_new_data ${MAX_DATA_TO_GENERATE} \
  --data_dir python \
  --tag python

train[:150000]
Parameter 'fn_kwargs'={'args': Args(seed_code_start_index=0, max_new_data=150, continue_from=None, seed=976, temperature=0.0, model='gpt-3.5-turbo-0125', model_max_tokens=8192, max_new_tokens=2500, min_lines=1, max_lines=15, chunk_size=1000, dataset_name='Nan-Do/code-search-net-python', data_dir='python', max_considered_data=150000, tag='python')} of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map: 100% 150000/150000 [00:05<00:00, 29488.85 examples/s]
Map: 100% 150000/150000 [00:49<00:00, 3043.63 examples/s]
Saving to data-python-ccfa1-0_150-20240319_025925.json

In [ ]:
!python src/magicoder/clean_data_scot.py --data_files ${PATH_TO_DATA_FILE} --output_file ${CLEANING_OUTPUT_PATH}

In [ ]:
!python -m magicoder.decontamination.find_substrings_scot \
  --dataset_name "json" \
  --output_file ${DECONTAM_OUTPUT_PATH} \
  --output_dir ${OUTPUT_DIR} \
  --columns problem solution \
  --data_files ${PATH_TO_DATA_FILE}

In [ ]:
!python src/magicoder/preprocess_data_scot.py \
  --dataset_path json \
  --data_files /content/magicoder/decontaminated.jsonl \
  --output_file ${PREPROCESS_OUTPUT_PATH} \
  --key src-instruct